# Введение в машинное обучение. Проектная работа. 
# Анализ поведения клиентов оператора мобильной связи «Мегалайн». 

## Описание проекта.

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф.

Задачи исследования:

* построение моделей для решения задачи классификации клиентов по тарифам.
* сравнение моделей и выбор одной с наилучшим значением метрики accuracy (доля правильных ответов) не менее 0.75.
* проверка модели на адекватность.

## Шаг 1. Откроем файл с данными и изучим общую инфоормацию.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split

In [ ]:
users = pd.read_csv('users_behavior.csv')
users.info()

### Описание данных
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
* сalls — количество звонков,
* minutes — суммарная длительность звонков в минутах,
* messages — количество sms-сообщений,
* mb_used — израсходованный интернет-трафик в Мб,
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).


### Вывод:
В таблице пять колонок и информация о поведении 3214 пользователях за месяц. Хороший набор данных без пропусков и некорректных значений.

## Шаг 2. Разделим исходные данные на обучающую, валидационную и тестовую выборки.

Нам необходимо выделить среди наших данных три выборки - обучающую, валидационную и тестовую.

Мы возьмем их в соотношениях 3 : 1 : 1.

Для этого воспользуемся функцией train_test_split 2 раза, кроме этого, сделаем стратификацию, чтобы доли обоих значений целевого признака во всех выборках были равны.

In [ ]:
target = users['is_ultra']
features = users.drop('is_ultra', axis=1)

features_train, features_valid_test, target_train, target_valid_test = train_test_split(features, 
                                        target, test_size=0.4, random_state=1, stratify = target)

features_valid, features_test, target_valid, target_test = train_test_split(features_valid_test, 
                        target_valid_test, test_size=0.5, random_state=1, stratify=target_valid_test)

Проверим, представительство классов в выборках.

In [ ]:
for i in [target_train, target_valid, target_test]:
    display(i.value_counts(), i.value_counts(normalize=True))

Заметно, что стратификация прошла успешно и доли классов во всех трех выборках одинаковы.

Итак, выборки сформированы в соотношении 3 : 1 : 1, признакам присвоены имена. Можно приступать к исследованию моделей.

## Шаг 3. Исследование моделей.

Прежде чем переходить к изучению моделей, создадим функцию для проверки точности модели на гиперпараметрах по умолчанию.

In [ ]:
def default_check(model_type, f_train, t_train, f_valid, t_valid):
    model = model_type(random_state=1)
    model.fit(f_train, t_train)
    model_predictions = model.predict(f_valid)
    return accuracy_score(t_valid, model_predictions)

### Дерево решений
Первой рассмотренной моделью будет алгоритм дерева решений.

Вызовем для него проверку по умолчанию.

In [ ]:
default_check(DecisionTreeClassifier, features_train, target_train, features_valid, target_valid)


Получается, что на валидационной выборке наша модель показала точность чуть выше 70%, что ниже требуемого заданием порога в 75%.

Посмотрим, приведет ли изменение гиперпараметров к повышению точности.

Перебирать различные значения гиперпараметров будем во вложенных циклах.

Для модели дерева решения попробуем менять следующие параметры (здесь и далее описание гиперпараметров приведено как попытка перевода с английского из документаций соответствующих алгоритмов):

* максимальная глубина дерева;
* минимальное количество объектов в листе - он будет запрещать создавать лист, в котором слишком мало объектов обучающей выборки, по умолчанию = 1;
* минимальное количество примеров для разделения - будет запрещать создавать узлы, в которые попадает слишком мало объектов обучающей выборки, по умолчанию = 2.
Перед выполнением цикла создадим новые переменные, которые будет сохранять информацию о гиперпараметрах лучшей по точности модели.

После обучения модели будем проверять ее на валидационной выборке.

In [ ]:
best_tree_model = None
best_tree_result = 0
best_depth = 0
best_samples_leaf = 0
best_samples_split = 0

for depth in range(1,10):
    for leaf in range(1,20):
        for split in range(2,10):
            tree_model = DecisionTreeClassifier(random_state=1, max_depth=depth, 
                                min_samples_leaf=leaf, min_samples_split=split)
            
            tree_model.fit(features_train, target_train)
            tree_predictions = tree_model.predict(features_valid)
            result = accuracy_score(target_valid, tree_predictions)
            
            if result > best_tree_result:
                
                best_tree_model = tree_model
                best_tree_result = result
                best_depth = depth
                best_samples_leaf = leaf
                best_samples_split = split

Выведем точность и значения гиперпараметров алгоритма лучшей модели.

In [ ]:
print(f'Точность наилучшей модели дерева решений: {best_tree_result:.2%}')
print(f'Максимальная глубина: {best_depth}')
print(f'Минимальное количество объектов в листе: {best_samples_leaf}')
print(f'Минимальное количество примеров для разделения: {best_samples_split}')

Получается, что точность модели выросла более чем на 10%, по сравнению с настройками по умолчанию.

Мы видим, что наше лучшее дерево достигло глубины в 7 уровней. Значение по умолчанию для этого гиперпараметра отсутствует - узлы расширяются до тех пор, пока все листья - чистые или содержат меньше, чем минимальное количество примеров для разделения.

Кроме того, в лучшей модели минимальное количество объектов выборки, попадающих в лист - 6.

Минимальное количество примеров для разделения осталось значением "по умолчанию".

### Случайный лес
Далее рассмотрим модель случайного леса.

Проверим ее на гиперпараметрах по умолчанию.

In [ ]:
default_check(RandomForestClassifier, features_train, target_train, features_valid, target_valid)

Модель сразу же выдала точность выше 78%.

Посомтрим, можно ли ее повысить, изменяя гиперпараметры.

В качестве изменяемых гиперпараметров возьмем следующие:

* n_estimators - количество деревьев в лесу;
* максимальная глубина дерева;
* максимальное количество листьев - то есть конечных ячеек.
* Выполним цикл для случайного леса.

In [ ]:
best_forest_model = None
best_forest_result = 0
best_depth_for_forest = 0
best_estimators = 0
best_max_leaf_nodes = 0

for est in range(1,10):
    for depth in range(1,10):
        for node in range(2,10):
            forest_model = RandomForestClassifier(random_state=1, max_depth=depth, 
                                n_estimators=est,max_leaf_nodes=node)
            
            forest_model.fit(features_train, target_train)
            forest_predictions = forest_model.predict(features_valid)
            result = accuracy_score(target_valid, forest_predictions)
            
            if result > best_forest_result:
                
                best_forest_model = forest_model
                best_forest_result = result
                best_depth_for_forest = depth
                best_estimators = est
                best_max_leaf_nodes = node

Выведем значения точности и гиперпараметров.

In [ ]:
print(f'Точность наилучшей модели случайного леса: {best_forest_result:.2%}')
print(f'Максимальная глубина: {best_depth_for_forest}')
print(f'Количество деревьев: {best_estimators}')
print(f'Максимальное количество листьев: {best_max_leaf_nodes}')

Итак, лучшая модель случайного леса насчитывает в себе 4 дерева.

Нам незначительно удалось повысить показатель точности, изменняя гиперпараметры.

То есть, алгоритм обучил 4 независимых друг от друга случайных деревьев, а потом принял решение на основе голосования.

Максимальная глубина - 3, у этой модели уровней у дерева меньше, чем у предыдущей.

Кроме того, получилось, что у лучшей модели число конечных ячеек (листьев) ограничено семью. (В этих ячейках назначаются классы).

### Логистическая регрессия
Далее рассмотрим алгоритм логистической регрессии.

Выполним для него проверку по умолчанию.

In [ ]:
default_check(LogisticRegression, features_train, target_train, features_valid, target_valid)

Точность модели на валидационной выборке - около 75%.

Посмотрим, удастся ли его повысить меняя гиперпараметры.

Попробуем менять следующие:

включение константы (пересечения с осью y) в функцию (по умолчанию=True);
выбор алгоритма для решения задачи оптимизации (по умолчанию lbfgs - метод с ограничением использования памяти алгоритма Бройдена — Флетчера — Гольдфарба — Шанно )
попробуем разное количество итераций для алгоритма оптимизации - при малых значений модель предупреждала об ошибке.

In [ ]:
best_LR_model = None
best_LR_result = 0
best_intercept = 0
best_solver = None
best_iter = 0
best_class = None

for intercept in [True, False]:
    for solver in ['lbfgs', 'liblinear', 'sag', 'saga']:
        for iterations in range(4000, 7000, 1500):
             
                    LR_model = LogisticRegression(random_state=1, fit_intercept=intercept,
                                             solver=solver, max_iter=iterations)
            
                    LR_model.fit(features_train, target_train)
                    LR_predictions = LR_model.predict(features_valid)
                    result = accuracy_score(target_valid, LR_predictions)
            
                    if result > best_LR_result:
                        best_LR_model = LR_model
                        best_LR_result = result
                        best_intercept = intercept
                        best_solver = solver
                        best_iter = iterations

In [ ]:
print(f'Точность наилучшей логистической регрессии: {best_LR_result:.2%}')
print(f'Включаем ли константу в функцию: {best_intercept}')
print(f'Алгоритм для решения задачи оптимизации: {best_solver}')
print(f'Максимальное количество итераций для алгоритма оптимизации: {best_iter}')

Итак, получается, что точность модели после манипуляций на валидационной выборке незначительно выросла.

Включение константы в функцию в лучшей модели осталось значением по умолчанию (включаем).

Алгоритм оптимизации остался также по умолчанию.

А количество итераций, потребовавшееся, чтобы модель работала без предупреждений - 4000.


### Вывод
Итак, мы рассмотрели три алгоритма обучения, для каждого из которых меняли гиперпараметры (в количестве трех штук).

В целом, можно сказать, что изменение гиперпараметров положительно сказалось на повышении точности моделей на валидационной выборке, по сравнению с точностью при использовании алгоритмами гиперпараметров по умолчанию.

Для дерева решений рост точности был заметным. Для случайного леса - более слабым. Для логистической регрессии - почти незаметным.

В целом можно сказать, что затратив время на подбор лучших гиперпараметров, нам удалось добиться роста точности моделей на валидационной выборке.

## Шаг 4. Проверим качество модели на тестовой выборке.

Напишем функцию проверки точности, которая принимает модели и тестовую выборку.

In [ ]:
def test_accuracy(model, features, target):
    predictions = model.predict(features)
    return accuracy_score(target, predictions)

In [ ]:
models = {'Дерево решений': best_tree_model, 'Случайный лес' : best_forest_model, 
          'Логистическая регрессия': best_LR_model}
for name, model in models.items():
    print(f'Точность модели {name} на тестовой выборке: {test_accuracy(model, features_test, target_test):.2%}')

Далее воспользуемся метрикой recall_score, чтобы узнать, какую долю каждого класса наши модели предсказали правильно.

In [ ]:
from sklearn.metrics import recall_score
for name, model in models.items():
    print(f'recall_score {name} на тестовой выборке: {recall_score(target_test, model.predict(features_test), average=None)}')

### Вывод
Итак, мы решали задачу классификации абонентов, исходя из того, сколько звонков делают, сколько минут они говорят, сколько сообщений отправляют и сколько мегабайтов интернета в месяц они тратят, зная пользуются ли они тарифом ultra, либо тарифом smart.

Все данные мы разбили на три выборки - обучающую, валидационную и тестовую в отношении 3 : 1 : 1.

Модели мы обучали тремя алгоритмами обучения - деревом решений, случайным лесом и логистической регрессией.

Все модели после перебора индивидуальных гиперпараметров показали повышение точности на валидационной выборке по сравнению с гиперпараметрами по умолчанию.

На тестовой выборке Дерево решений и случайный лес показали точность около 80% соответственно. В то время как логистическая регрессия чуть перешагнула за границу в 75%.

Также интересны были результаты работы recall_score. Оказалось, что случайный лес хуже всех прогнозирует абонентов тарифа "Смарт" (93%), но лучше всех - абонентов 'ultra' (почти 50%). Дерево решений имеет похожие результаты, но немного лучше по "смарт" и существенно хуже по "ультра".

Логистическая регрессия отлично предсказывает абонентов "смарт" и только четверть абонентов "ультра".

## Шаг 5. Провериим модели на вменяемость.
(дополнительное задание)

Итак, sanity-check или "проверка на вменяемость" подразумевает относительно простой тест для определения, может ли быть модель адекватно применяемой.

Для того, чтобы попытаться провести такую проверку для наших моделей обратимся к значениям целевого признака по датасету.

In [ ]:
users['is_ultra'].mean()

Значит, в нашем наборе данных около 69.5% абонентов пользуются тарифом "смарт".

Следовательно, если мы предположим, что наши модели будут всегда предсказывать тариф "смарт", то точность моделей была бы на уровне 69.5%. Но у каждой из трех моделей после подбора лучших гиперпараметров она превышает это значение.

In [ ]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_train, target_train)
dummy_clf.score(dummy_clf.predict(features_valid), target_valid)

### Вывод:
Таким обазом, на мой взгляд, модели прошли проверку на вменяемость.

## Выводы:

В рамках исследования с целью построения модели для задачи бинарной классификации клиентов оператора «Мегалайн» по тарифным планам «Смарт» и «Ультра» выполнены следующие основные этапы:

* проведена предварительная оценка и предобработка данных
* выполнено разбиение (с сохранением пропорции в целевом атрибуте is_ultra) исходных данных случайным образом на обучающую, валидационную и тестовую выборки
* выполнен расчет дополнительных атрибутов для последующего построения и сравнения моделей
* определен целевой признак и сформирован словарь обучающих признаков для построения моделей
* для каждого набора обучающих признаков проведена оценка гиперпараметров моделей дерева решений и случайного леса и определены модели с наиболее высоким значением метрики accuracy, вычисленным на валидационной выборке
* для каждого набора обучающих признаков построена модель логистической регрессии и также определено наиболее высокое значение метрики accuracy
* сформированный перечень моделей с наиболее высокими значениями метрики accuracy подготовлен для последующей проверки на тестовой выборке.

По результатам проверки на тестовой выборке получены следующие результаты:

* всем отобранным моделям удалось достичь требуемого значения метрики accuracy равного 0.75
* наивысшее значение accuracy равное 0.80 у модели случайного леса с числом деревьев 4 и максимальной глубиной 3 и модель дерева решений с глубиной 7 и количеством объектов в листе 6.
На тестовой выборке Дерево решений и случайный лес показали точность около 80% соответственно. В то время как логистическая регрессия чуть перешагнула за границу в 75%.

Также проведена проверка модели на адекватность, т.е. сравнение полученного значения метрики accuracy с простыми способами определения тарифа:

* с вероятностью случайного угадывания тарифа пользователя - определена равной 0.69 на основе определения доли абонентов тарифа «Смарт» в исходном репрезентативной выборке;
* Полученное в результате исследования значение метрики accuracy (0.80) выше указанных вероятностей, соответственно, модель улучшает случайные метрики, т.е. адекватна.